In [1]:
import os
cwd = os.getcwd()

In [2]:
def reverse_complement(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'} 
    bases = list(seq) 
    for base in bases:
        if base not in complement.keys():
            print "not DNA"  
    complement_bases = [complement[base] for base in bases] 
    complement_seq = ''.join(complement_bases)
    reverse_complement_seq = complement_seq[::-1] 
    return reverse_complement_seq


def gff_plus_strand(gff_file):   
    # a function to get the sequence of E coli genome from GFF file
    with open(gff_file) as fh:
        plus_strand = ''
        flag = 0
        for line in fh:
            if line[0] == ">":
                flag = 1
                continue
            if flag == 0:
                continue
            plus_strand = plus_strand + line[:-1]
    return plus_strand 


def free_align(testing_seq,standard_seq,b,l,temp_file_name,cwd):
    import os
        
    os.chdir(cwd)
    #os.chdir('free2bind') # move just to run free_align
    ###### open a temporary file 
    tempo_file = temp_file_name+'_free_align.txt'
    #f = open(tempo_file,'w')
    
    ###### Calculation of deltaG and write whole output of free_align on temporary file
    free_align_commandline = 'perl free_align.pl -b '+b+' -l '+l+' '+testing_seq+' '+standard_seq+' > '+tempo_file
    os.chdir('free2bind') # move just to run free_align
    os.system(free_align_commandline)
    
    ###### close the temporary file 
    #f.close()
    
    
    ###### get the delta-G value to from the temporary file 
    with open(tempo_file) as deltaG_h:
        for dGline in deltaG_h:
            if dGline[0:5] == "Delta":
                dGline_list = dGline.split()
                energy = dGline_list[8]
                return energy
    os.chdir(cwd) # get back to the directory of genome anlysis

In [3]:
gff = 'coli.gff'
plus_strand_sequence = gff_plus_strand(gff)

In [4]:
import pickle
import numpy as np
from pandas import Series, DataFrame
import pandas as pd

os.chdir(cwd)

SDASD_dict = {}
with open("coli.gff") as fh:
    genecounts=0
    GeneFlag=0
    for line in fh:
        listedline = line.split()
        if listedline[0] == "###":
            break
        if line[0] == "#":
            continue
        if listedline[2] == 'gene':
            genecounts+=1
            listedattributes = listedline[8].split(";")  
            genename = listedattributes[2].replace("Name=","")
            if listedline[6] == "+":
                minus15 = int(listedline[3])-16
                minus5 = int(listedline[3])-6
                rbs = plus_strand_sequence[minus15:minus5]
            else:
                minus5comp = int(listedline[4])+5
                minus15comp = int(listedline[4])+15
                rbscomp = plus_strand_sequence[minus5comp:minus15comp]
                rbs = reverse_complement(rbscomp)
                
            SDASD = free_align(rbs,'uCCUCCa','0','0','temp_file',cwd) 
            
            SDASD_dict[genename]=[]
            SDASD_dict[genename].append(rbs)
            SDASD_dict[genename].append(SDASD)   
os.chdir(cwd)

In [5]:
SDASD_dict

{'rbfA': ['TCAGGAGAAT', '-5.982865'],
 'xapA': ['AGAAAAAGGA', '-4.10793'],
 'xapB': ['GACAAGGAAA', '-3.90793'],
 'yfjK': ['GGTGAGAGAG', '-3.323445'],
 'rhtC': ['CCGCGAGTCA', '-0.359835'],
 'abrB': ['ATGAGGGAGC', '-5.7311'],
 'xapR': ['ATGGAGCACT', '-3.60793'],
 'speF': ['ATAAAGAGAT', '-0.359835'],
 'speG': ['ATGTAAGGAC', '-3.90793'],
 'speD': ['GGAGAAGATA', '-4.80793'],
 'speE': ['AGGAGGTATC', '-11.881535'],
 'speB': ['AATAGGAGTC', '-5.882865'],
 'speC': ['AACGGGTTTG', '-3.646615'],
 'speA': ['AATTTGAGGT', '-5.39508'],
 'ybcL': ['AACAAAGAGG', '-4.10793'],
 'ybcM': ['AAGTAGGGTG', '-4.246615'],
 'ybcN': ['CTGCGGAGGG', '-6.856025'],
 'ybcO': ['CAGCAAGGAG', '-6.182865'],
 'ybcH': ['GGAGTGCACT', '-4.80793'],
 'ybcI': ['GCCGAAGCGA', '1.5151'],
 'ybcJ': ['CACAGGATGA', '-3.90793'],
 'ybcK': ['TTAGCAGGTG', '-4.19838'],
 'frdA': ['ATCTGGAGGA', '-6.856025'],
 'ybcF': ['ATTACAAGGA', '-4.10793'],
 'frdC': ['GCTAAGGAGT', '-5.982865'],
 'frdD': ['GGTAAGGAGC', '-5.982865'],
 'yeaW': ['GATAAAGGTA', '-4